In [1]:
using DifferentiableFactorizations, LinearAlgebra, BenchmarkTools

In [2]:
function qr_conditions_dev(A, x)
    (; Q, R) = x
    m, n = size(A, 1), size(A, 2)
    res = similar(A, m + n, n)
    res_A = view(res, 1:m, :)
    res_R = view(res, m+1:m+n, :)
    copyto!(res_A, A)
    fill!(res_R, 0)
    for i in 1:min(m, n)
      res_R[i, i] = 1
    end
    for i in 1:m
      for j in 1:n
        for k in 1:n
          res_A[i, j] -= Q[i, k] * R[k, j]
          if i == 1 && j < k
            res_R[k, j] += R[k, j]
          end
          if j <= k
            res_R[j, k] -= Q[i, k] * Q[i, j]
          end
        end
      end
    end
    return vec(res)
end


qr_conditions_dev (generic function with 1 method)

In [3]:
function qr_conditions(A, x)
    (; Q, R) = x
    return vcat(
      vec(UpperTriangular(Q' * Q) + LowerTriangular(R) - I - Diagonal(R)),
      vec(Q * R - A),
    )
end

qr_conditions (generic function with 1 method)

In [4]:
@benchmark qr_conditions_dev(A, x) setup=begin
    A = rand(5, 3)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 10000 samples with 191 evaluations.
 Range (min … max):  516.885 ns …  25.518 μs  ┊ GC (min … max): 0.00% … 96.82%
 Time  (median):     614.196 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   733.494 ns ± 814.352 ns  ┊ GC (mean ± σ):  3.99% ±  3.61%

  █▇▆▁                                                           
  ████▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  517 ns           Histogram: frequency by time         1.37 μs <

 Memory estimate: 336 bytes, allocs estimate: 3.

In [5]:
@benchmark qr_conditions(A, x) setup=begin
    A = rand(5, 3)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.259 μs … 490.767 μs  ┊ GC (min … max): 0.00% … 98.59%
 Time  (median):     2.236 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.479 μs ±   6.808 μs  ┊ GC (mean ± σ):  3.77% ±  1.39%

  █▂                                                           
  ███▅▄▃▃▃▃▃▄▄▄▄▅▅▆▆▅▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.26 μs         Histogram: frequency by time           7 μs <

 Memory estimate: 1.58 KiB, allocs estimate: 14.

In [6]:
@benchmark qr_conditions_dev(A, x) setup=begin
    A = rand(100, 60)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 1593 samples with 1 evaluation.
 Range (min … max):  1.937 ms …   4.357 ms  ┊ GC (min … max): 0.00% … 42.40%
 Time  (median):     2.183 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.291 ms ± 320.644 μs  ┊ GC (mean ± σ):  0.24% ±  2.46%

    ▃▇██▆▇▅▅▄▅ ▂                                               
  ▂▇██████████▇█▇▅▅▅▆▄▄▄▄▃▄▄▃▂▂▃▂▂▂▂▃▃▂▃▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▂▁▁▁▁▂ ▃
  1.94 ms         Histogram: frequency by time        3.36 ms <

 Memory estimate: 75.12 KiB, allocs estimate: 4.

In [7]:
@benchmark qr_conditions(A, x) setup=begin
    A = rand(100, 60)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 5273 samples with 1 evaluation.
 Range (min … max):  111.835 μs …   3.115 ms  ┊ GC (min … max):  0.00% … 92.59%
 Time  (median):     157.663 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   189.266 μs ± 231.089 μs  ┊ GC (mean ± σ):  11.64% ±  9.07%

  ██▇▄▂▁▂ ▁▁                                                    ▁
  ███████████▇▅▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▄ █
  112 μs        Histogram: log(frequency) by time       1.93 ms <

 Memory estimate: 338.61 KiB, allocs estimate: 23.

In [8]:
@benchmark qr_conditions_dev(A, x) setup=begin
    A = rand(100, 3)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   9.107 μs …  1.057 ms  ┊ GC (min … max): 0.00% … 93.01%
 Time  (median):     11.044 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.261 μs ± 27.971 μs  ┊ GC (mean ± σ):  6.41% ±  3.03%

   █▆                                                          
  ▁██▅▃▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  9.11 μs         Histogram: frequency by time        24.4 μs <

 Memory estimate: 2.64 KiB, allocs estimate: 3.

In [9]:
@benchmark qr_conditions(A, x) setup=begin
    A = rand(100, 3)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.343 μs … 628.807 μs  ┊ GC (min … max):  0.00% … 91.88%
 Time  (median):     5.460 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.291 μs ±  24.479 μs  ┊ GC (mean ± σ):  15.26% ±  4.51%

     ▃██▅▄▁                                                    
  ▁▂▅██████▆▅▄▃▃▃▃▂▃▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.34 μs         Histogram: frequency by time        12.8 μs <

 Memory estimate: 8.55 KiB, allocs estimate: 14.

In [10]:
@benchmark qr_conditions_dev(A, x) setup=begin
    A = rand(100, 100)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 635 samples with 1 evaluation.
 Range (min … max):  5.102 ms … 17.203 ms  ┊ GC (min … max): 0.00% … 19.03%
 Time  (median):     5.994 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.456 ms ±  1.404 ms  ┊ GC (mean ± σ):  0.99% ±  4.25%

   ▂█▆▆▂▃                                                     
  ▆███████▇▆▆▆▅▅▃▃▄▄▃▄▅▅▅▄▄▃▂▁▂▂▁▁▂▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  5.1 ms         Histogram: frequency by time        13.4 ms <

 Memory estimate: 156.38 KiB, allocs estimate: 4.

In [11]:
@benchmark qr_conditions(A, x) setup=begin
    A = rand(100, 100)
    x = diff_qr(A)
end

BenchmarkTools.Trial: 2678 samples with 1 evaluation.
 Range (min … max):  296.538 μs …  16.544 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     456.152 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   526.096 μs ± 454.992 μs  ┊ GC (mean ± σ):  6.41% ± 9.49%

  ▄██▇▇▇▇▆▄▂▁       ▁▁                                          ▂
  ████████████▆▇█▇▇███▇▇▅▅▄▁▄▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▄▁▁▄▁▁▆▅▄▁▅▁▆▅ █
  297 μs        Histogram: log(frequency) by time       2.58 ms <

 Memory estimate: 782.70 KiB, allocs estimate: 23.